# fusion_engine

Fusion engine for combining image detections and GPS anomalies to generate poaching alerts.

In [ ]:
import pandas as pdimport numpy as npfrom datetime import datetime, timedeltafrom utils import calculate_distance, save_dataframe

## Code

In [ ]:
Fusion engine for combining image detections and GPS anomalies to generate poaching alerts.class FusionEngine:    def __init__(self, proximity_threshold=500):        Initialize the fusion engine.        Args:            proximity_threshold: Distance threshold in meters for proximity alerts        self.proximity_threshold = proximity_threshold        self.alerts = []    def load_data(self, image_detections_path, gps_anomalies_path):        Load image detections and GPS anomalies data.        Args:            image_detections_path: Path to image detections CSV            gps_anomalies_path: Path to GPS anomalies CSV        Returns:            Tuple of (image_detections_df, gps_anomalies_df)        print("Loading detection and anomaly data...")        try:            image_detections_df = pd.read_csv(image_detections_path)            image_detections_df['timestamp'] = pd.to_datetime(image_detections_df['timestamp'], errors='coerce')        except Exception as e:            print(f"Error loading image detections: {e}")            image_detections_df = pd.DataFrame()        try:            gps_anomalies_df = pd.read_csv(gps_anomalies_path)            gps_anomalies_df['timestamp'] = pd.to_datetime(gps_anomalies_df['timestamp'])        except Exception as e:            print(f"Error loading GPS anomalies: {e}")            gps_anomalies_df = pd.DataFrame()        print(f"Loaded {len(image_detections_df)} image detections")        print(f"Loaded {len(gps_anomalies_df)} GPS points with {gps_anomalies_df['is_anomaly'].sum()} anomalies")        return image_detections_df, gps_anomalies_df    def check_proximity_alerts(self, image_detections_df, gps_anomalies_df):        Check for proximity between human/vehicle detections and animal anomalies.        Args:            image_detections_df: DataFrame with image detections            gps_anomalies_df: DataFrame with GPS anomalies        Returns:            List of proximity alerts        print("Checking proximity between detections and anomalies...")        proximity_alerts = []        if image_detections_df.empty or gps_anomalies_df.empty:            return proximity_alerts        # Get anomalous GPS points        anomalous_points = gps_anomalies_df[gps_anomalies_df['is_anomaly']].copy()        if anomalous_points.empty:            return proximity_alerts        # Check each detection against each anomaly        for _, detection in image_detections_df.iterrows():            detection_lat = detection['detection_lat']            detection_lon = detection['detection_lon']            detection_time = detection['timestamp']            for _, anomaly in anomalous_points.iterrows():                anomaly_lat = anomaly['latitude']                anomaly_lon = anomaly['longitude']                anomaly_time = anomaly['timestamp']                # Calculate distance                distance = calculate_distance(detection_lat, detection_lon, anomaly_lat, anomaly_lon)                # Check if within proximity threshold                if distance <= self.proximity_threshold:                    # Check time proximity (within 2 hours)                    time_diff = abs((detection_time - anomaly_time).total_seconds())                    if time_diff <= 7200:  # 2 hours in seconds                        # Determine alert level based on detection type and distance                        if detection['class'] == 'person' and distance <= 200:                            alert_level = 'High'                        elif detection['class'] in ['car', 'truck'] and distance <= 300:                            alert_level = 'High'                        elif distance <= 100:                            alert_level = 'High'                        elif distance <= 300:                            alert_level = 'Medium'                        else:                            alert_level = 'Low'                        proximity_alerts.append({                            'alert_type': 'Proximity Alert',                            'alert_level': alert_level,                            'latitude': (detection_lat + anomaly_lat) / 2,  # Midpoint                            'longitude': (detection_lon + anomaly_lon) / 2,                            'timestamp': max(detection_time, anomaly_time),                            'detection_id': detection['image_id'],                            'animal_id': anomaly['animal_id'],                            'distance_meters': distance,                            'detection_class': detection['class'],                            'detection_confidence': detection['confidence'],                            'anomaly_score': anomaly['anomaly_score'],                            'description': f"{detection['class']} detected {distance:.0f}m from {anomaly['animal_id']} anomaly"                        })        print(f"Generated {len(proximity_alerts)} proximity alerts")        return proximity_alerts    def check_zone_alerts(self, image_detections_df):        Check for detections in core wildlife zones.        Args:            image_detections_df: DataFrame with image detections        Returns:            List of zone alerts        print("Checking for detections in core wildlife zones...")        zone_alerts = []        if image_detections_df.empty:            return zone_alerts        # Define core wildlife zones (example coordinates)        core_zones = [            {'name': 'Core Zone 1', 'lat': -1.2921, 'lon': 36.8219, 'radius': 1000},            {'name': 'Core Zone 2', 'lat': -1.3000, 'lon': 36.8300, 'radius': 800},        ]        for _, detection in image_detections_df.iterrows():            detection_lat = detection['detection_lat']            detection_lon = detection['detection_lon']            for zone in core_zones:                distance = calculate_distance(                    detection_lat, detection_lon,                     zone['lat'], zone['lon']                )                if distance <= zone['radius']:                    # Determine alert level based on detection type                    if detection['class'] == 'person':                        alert_level = 'High'                    elif detection['class'] in ['car', 'truck']:                        alert_level = 'Medium'                    else:                        alert_level = 'Low'                    zone_alerts.append({                        'alert_type': 'Zone Violation',                        'alert_level': alert_level,                        'latitude': detection_lat,                        'longitude': detection_lon,                        'timestamp': detection['timestamp'],                        'detection_id': detection['image_id'],                        'animal_id': None,                        'distance_meters': distance,                        'detection_class': detection['class'],                        'detection_confidence': detection['confidence'],                        'anomaly_score': None,                        'description': f"{detection['class']} detected in {zone['name']}"                    })        print(f"Generated {len(zone_alerts)} zone violation alerts")        return zone_alerts    def check_temporal_alerts(self, gps_anomalies_df):        Check for temporal patterns in GPS anomalies.        Args:            gps_anomalies_df: DataFrame with GPS anomalies        Returns:            List of temporal alerts        print("Checking for temporal anomaly patterns...")        temporal_alerts = []        if gps_anomalies_df.empty:            return temporal_alerts        # Group anomalies by animal and time        anomalous_points = gps_anomalies_df[gps_anomalies_df['is_anomaly']].copy()        if anomalous_points.empty:            return temporal_alerts        # Check for multiple anomalies in short time periods        for animal_id in anomalous_points['animal_id'].unique():            animal_anomalies = anomalous_points[anomalous_points['animal_id'] == animal_id].copy()            animal_anomalies = animal_anomalies.sort_values('timestamp')            # Check for clustering of anomalies (3+ anomalies within 6 hours)            for i in range(len(animal_anomalies) - 2):                current_time = animal_anomalies.iloc[i]['timestamp']                future_anomalies = animal_anomalies[                    (animal_anomalies['timestamp'] >= current_time) &                     (animal_anomalies['timestamp'] <= current_time + timedelta(hours=6))                ]                if len(future_anomalies) >= 3:                    # Calculate center of anomaly cluster                    cluster_lat = future_anomalies['latitude'].mean()                    cluster_lon = future_anomalies['longitude'].mean()                    temporal_alerts.append({                        'alert_type': 'Temporal Pattern',                        'alert_level': 'High',                        'latitude': cluster_lat,                        'longitude': cluster_lon,                        'timestamp': current_time,                        'detection_id': None,                        'animal_id': animal_id,                        'distance_meters': None,                        'detection_class': None,                        'detection_confidence': None,                        'anomaly_score': future_anomalies['anomaly_score'].mean(),                        'description': f"Multiple anomalies detected for {animal_id} within 6 hours"                    })        print(f"Generated {len(temporal_alerts)} temporal pattern alerts")        return temporal_alerts    def generate_alerts(self, image_detections_path, gps_anomalies_path):        Generate all types of poaching alerts.        Args:            image_detections_path: Path to image detections CSV            gps_anomalies_path: Path to GPS anomalies CSV        Returns:            DataFrame with all generated alerts        print("Generating poaching alerts...")        # Load data        image_detections_df, gps_anomalies_df = self.load_data(image_detections_path, gps_anomalies_path)        all_alerts = []        # Generate different types of alerts        proximity_alerts = self.check_proximity_alerts(image_detections_df, gps_anomalies_df)        zone_alerts = self.check_zone_alerts(image_detections_df)        temporal_alerts = self.check_temporal_alerts(gps_anomalies_df)        all_alerts.extend(proximity_alerts)        all_alerts.extend(zone_alerts)        all_alerts.extend(temporal_alerts)        # Create alerts DataFrame        if all_alerts:            alerts_df = pd.DataFrame(all_alerts)            # Add alert IDs            alerts_df['alert_id'] = [f"ALERT_{i:06d}" for i in range(len(alerts_df))]            # Sort by timestamp and alert level            alerts_df = alerts_df.sort_values(['alert_level', 'timestamp'], ascending=[False, True])            # Save alerts            save_dataframe(alerts_df, 'poaching_alerts.csv', 'output')            print(f"Generated {len(alerts_df)} total alerts")            print(f"Alert breakdown by level:")            print(alerts_df['alert_level'].value_counts())            print(f"Alert breakdown by type:")            print(alerts_df['alert_type'].value_counts())        else:            print("No alerts generated")            alerts_df = pd.DataFrame(columns=[                'alert_id', 'alert_type', 'alert_level', 'latitude', 'longitude',                'timestamp', 'detection_id', 'animal_id', 'distance_meters',                'detection_class', 'detection_confidence', 'anomaly_score', 'description'            ])            save_dataframe(alerts_df, 'poaching_alerts.csv', 'output')        return alerts_df    def get_alert_summary(self, alerts_df):        Get summary statistics for generated alerts.        Args:            alerts_df: DataFrame with alerts        Returns:            Dictionary with summary statistics        if alerts_df.empty:            return {                'total_alerts': 0,                'high_priority_alerts': 0,                'medium_priority_alerts': 0,                'low_priority_alerts': 0,                'proximity_alerts': 0,                'zone_violations': 0,                'temporal_patterns': 0            }        return {            'total_alerts': len(alerts_df),            'high_priority_alerts': len(alerts_df[alerts_df['alert_level'] == 'High']),            'medium_priority_alerts': len(alerts_df[alerts_df['alert_level'] == 'Medium']),            'low_priority_alerts': len(alerts_df[alerts_df['alert_level'] == 'Low']),            'proximity_alerts': len(alerts_df[alerts_df['alert_type'] == 'Proximity Alert']),            'zone_violations': len(alerts_df[alerts_df['alert_type'] == 'Zone Violation']),            'temporal_patterns': len(alerts_df[alerts_df['alert_type'] == 'Temporal Pattern'])        }

## Test Code

In [ ]:
    # Test the fusion engine    engine = FusionEngine()    # Generate alerts    alerts = engine.generate_alerts('output/image_detections.csv', 'output/gps_anomalies.csv')    summary = engine.get_alert_summary(alerts)    print("Alert Summary:", summary)